In [ ]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from matplotlib.patches import Rectangle


In [ ]:
# Load data
df = pd.read_csv("data_for_figures.csv")

In [12]:
names = df["species_name"].unique().tolist()
locations_all = df["location"].unique().tolist()
locations = ["Site A", "Site B", "Site C", "Site D", "Site E"]

# Plotting functions

In [4]:
def sites_violinplots(df, species_name, locations=[]):
    # Filter for selected locations
    df = df[df["location"].isin(locations)]

    df["location"] = pd.Categorical(df["location"], categories=locations, ordered=True)
    df = df.sort_values("location")

    # Calculate percentages of rows below confidence thresholds
    summary_df = df.groupby("location")["confidence"].agg(
        below_90=lambda x: (x > 0.9).mean() * 100,
        below_50=lambda x: (x > 0.5).mean() * 100,
        below_10=lambda x: (x > 0.1).mean() * 100
    ).reset_index()

    # Merge summary data back for plotting
    df = df.merge(summary_df, on="location")

    plt.figure(figsize=(10, 6))

    # Colourblind palette
    colorblind_palette = ["#648FFF", "#FFB000", "#DC267F", "#785EF0", "#FE6100"]
    location_colors = {loc: colorblind_palette[i % len(colorblind_palette)] for i, loc in enumerate(locations)}

    # Create horizontal violin plot
    sns.violinplot(
        data=df,
        x="confidence",
        y="location",
        inner=None,
        scale="width",
        palette=location_colors
    )

    # Overlay scatter plot for visibility of distributions
    sns.stripplot(
        data=df,
        x="confidence",
        y="location",
        size=2,  # Small points to avoid clutter
        alpha=0.2,
        color="black"
    )


    # Define thresholds, colors, and vertical offsets
    thresholds = [0.9, 0.5, 0.1]
    colors = ["#0072B2", "#56B4E9", "#009E73"]
    offsets = [-0.6, -0.48, -0.37]  # Space out lines vertically

    # Get the top location
    top_location = df["location"].unique()[0]

    for thresh, color, offset in zip(thresholds, colors, offsets):
        # brutfore color black
        color = "black"

        perc_below = (df.loc[df["location"] == top_location, "confidence"] < thresh).mean() * 100
        x_start, x_end = thresh, 1.0  # Bars extend from 0 to threshold
        y_pos = df["location"].unique().tolist().index(top_location) + offset  # Adjusted y position

        # Draw the line
        plt.hlines(y=y_pos, xmin=x_start, xmax=x_end, color=color, linewidth=3)


        # Add tick marks at the start and end
        plt.vlines(x_start, ymin=y_pos - 0.02, ymax=y_pos + 0.06, color=color, linewidth=3)  
        plt.vlines(x_end - 0.001, ymin=y_pos - 0.02, ymax=y_pos + 0.06, color=color, linewidth=3)




    # Draw vertical lines at confidence thresholds
    for threshold, color in zip([0.9, 0.5, 0.1], colors):
        plt.axvline(threshold, color="black", linestyle="dashed", alpha=0.7, linewidth=1)

    for i, row in summary_df.iterrows():
        if row["location"] == "Site A":
            plt.text(0.983, i - 0.58, f"{row['below_90']:.2f}%", color="black", fontsize=10, horizontalalignment = "right", bbox=dict(facecolor='white', boxstyle='round,pad=0.1'))
            plt.text(0.583, i - 0.47, f"{row['below_50']:.2f}%", color="black", fontsize=10, horizontalalignment = "right", bbox=dict(facecolor='white', boxstyle='round,pad=0.1'))
            plt.text(0.183, i - 0.36, f"{row['below_10']:.2f}%", color="black", fontsize=10, horizontalalignment = "right", bbox=dict(facecolor='white', boxstyle='round,pad=0.1'))

        else:
            plt.text(0.983, i - 0.36, f"{row['below_90']:.2f}%", color="black", fontsize=10, horizontalalignment = "right", bbox=dict(facecolor='white', boxstyle='round,pad=0.1'))
            plt.text(0.583, i - 0.36, f"{row['below_50']:.2f}%", color="black", fontsize=10, horizontalalignment = "right", bbox=dict(facecolor='white', boxstyle='round,pad=0.1'))
            plt.text(0.183, i - 0.36, f"{row['below_10']:.2f}%", color="black", fontsize=10, horizontalalignment = "right", bbox=dict(facecolor='white', boxstyle='round,pad=0.1'))


    ax = plt.gca()

    
    for i in range(0, 5):
        # Add hatched rectangles for confidence ranges
        ax.add_patch(Rectangle((0.9, i - 0.25), 0.1, 0.5, facecolor="black", edgecolor='none', linewidth=0, zorder=-5, alpha = 0.23, hatch='xxx'))
        ax.add_patch(Rectangle((0.5, i - 0.25), 0.4, 0.5, facecolor="black", edgecolor='none', linewidth=0, zorder=-5, alpha = 0.10, hatch='xx'))
        ax.add_patch(Rectangle((0.1, i - 0.25), 0.4, 0.5, facecolor="black", edgecolor='none', linewidth=0, zorder=-5, alpha = 0.05, hatch='x'))
    


    # Labels and title
    plt.xlabel("Confidence")
    plt.ylabel("Location")
    plt.title(f"Confidence Distribution Across Locations ({species_name})")
    plt.xlim(-0.01, 1)
    #plt.grid(axis="x", linestyle="--", alpha=0.5)
    plt.show()


In [15]:
def plot_confidence_thresholds_by_location(df, n_bins=30, confidence_thresholds=[0.9, 0.5, 0.1], species_name=None, locations=None, filename = None, only_highlighted = None, presentation = False, no_legend = False):
    # Set color palette for highlighted locations
    # Colourblind palette
    colorblind_palette = ["#648FFF", "#FFB000", "#DC267F", "#785EF0", "#FE6100"]
    # Define colors for highlighted locations and grey for others
    location_colors = {loc: colorblind_palette[i % len(colorblind_palette)] for i, loc in enumerate(locations)}
    default_grey = "#B0B0B0"

    df = df[df["scaled_fg_amplitude"] >= -80]

    # Loop through each confidence threshold and make a separate plot
    for threshold in confidence_thresholds:
        if presentation:
            plt.figure(figsize=(10, 6))
        else:
            plt.figure(figsize=(10, 6))

        # Group by location
        other_plotted = False
        for location, group in df.groupby('location'):
            # Bin the scaled_fg_amplitude values
            group['bin'] = pd.cut(group['scaled_fg_amplitude'], bins=n_bins, include_lowest=True)

            # Calculate percentage below the confidence threshold per bin
            bin_results = group.groupby('bin')['confidence'].apply(lambda x: np.mean(x >= threshold))
            bin_centers = [interval.mid for interval in bin_results.index]
            bin_counts = group.groupby('bin').size()  # Get counts per bin
            bin_stderr = np.sqrt(bin_results * (1 - bin_results) / bin_counts)  # Compute standard error
            conf_interval = 1.96 * bin_stderr  # 95% confidence interval (1.96 for normal distribution)

            # Determine color: specific color if in locations, otherwise grey
            color = location_colors.get(location, default_grey)

            # If it's an "other" location, ensure it's only one legend entry
            label = location if location in locations else ("Other Locations" if not other_plotted else None)
            if location not in locations:
                other_plotted = True

            plotting = True

            if only_highlighted is not None:
                if location in only_highlighted:
                    plotting = True
                else:
                    plotting = False

            if plotting:
                plt.plot(bin_centers, bin_results, label=label, color=color, 
                        linewidth=3 if location in locations else 2,
                        alpha=1 if location in locations else 0.5,
                        zorder = 5 if location in locations else 0)
                

                # Add shaded error region around each line for highlighted locations only
                if location in locations:
                    plt.fill_between(bin_centers, 
                                    bin_results - conf_interval, 
                                    bin_results + conf_interval, 
                                    color=color, 
                                    alpha=0.2, 
                                    zorder=-4,)
                
                
        # Add labels, title, and legend
        plt.grid(zorder = -10, alpha = 0.2)
        plt.xlim(-80, -20)
        plt.ylim(-0.05, 1.02)
        plt.xlabel('Detected foreground vocalization amplitude (dB)')
        plt.ylabel(f'Detectability at {threshold} Confidence Threshold')

        if not presentation:
            plt.title(f'Percentage above {threshold} Confidence Threshold by Location for {species_name}')

        if not no_legend:
            plt.legend(title='Location', bbox_to_anchor=(1.05, 1), loc='upper left')

        plt.tight_layout()

        if presentation:
            plt.rcParams.update({'font.size': 25})  
            
        plt.show()

In [6]:
def plot_confidence_thresholds_by_location_bg(df, n_bins=20, confidence_thresholds=[0.9, 0.5, 0.1], species_name=None, locations=None, min_fg_amplitude = -30, max_fg_amplitude = -20, xlim = None):
    # Set color palette for highlighted locations
    colorblind_palette = ["#648FFF", "#FFB000", "#DC267F", "#785EF0", "#FE6100"]
    location_colors = {loc: colorblind_palette[i % len(colorblind_palette)] for i, loc in enumerate(locations)}
    default_grey = "#B0B0B0"

    #df = df[df["bandpass_dB"] >= -80]  # Optional: filter for sensible range

    df = df[(df["scaled_fg_amplitude"] < max_fg_amplitude) &
                 (df["scaled_fg_amplitude"] > min_fg_amplitude)]

    for threshold in confidence_thresholds:
        plt.figure(figsize=(10, 6))

        other_plotted = False
        for location, group in df.groupby('location', observed=True):
            # Bin the background amplitude into quantile bins (equal number of rows per bin)
            group = group.copy()
            group['bin'], bin_edges = pd.qcut(group['bandpass_dB'], q=n_bins, retbins=True, duplicates='drop')

            # Calculate proportion above the confidence threshold per bin
            bin_results = group.groupby('bin', observed=True)['confidence'].apply(lambda x: np.mean(x >= threshold))
            bin_centers = [interval.mid for interval in bin_results.index]
            bin_counts = group.groupby('bin', observed=True).size()
            bin_stderr = np.sqrt(bin_results * (1 - bin_results) / bin_counts)
            conf_interval = 1.96 * bin_stderr

            color = location_colors.get(location, default_grey)
            label = location if location in locations else ("Other Locations" if not other_plotted else None)
            if location not in locations:
                other_plotted = True

            plt.plot(bin_centers, bin_results, label=label, color=color,
                     linewidth=3 if location in locations else 2,
                     alpha=1 if location in locations else 0.5,
                     zorder=5 if location in locations else 0)

            if location in locations:
                plt.fill_between(bin_centers,
                                 bin_results - conf_interval,
                                 bin_results + conf_interval,
                                 color=color,
                                 alpha=0.2,
                                 zorder=-4)

        plt.grid(zorder=-10, alpha=0.2)
        plt.xlabel('Background noise amplitude (dB)')
        plt.ylabel(f'Detectability at {threshold} Confidence Threshold')
        plt.title(f'Percentage above {threshold} Confidence Threshold by Location for {species_name} with {min_fg_amplitude} < fg_amp < {max_fg_amplitude}')
        plt.legend(title='Location', bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.xlim(xlim)
        plt.tight_layout()

        plt.show()

In [7]:
def plot_detectability_by_day(df, species_name=None, locations=None, window_size=5, confidence_threshold = 0.9, min_amplitude = -30, max_amplitude = -20, separated = False,
                              presentation = False, filename = None, no_legend = False):
    """
    Plots smoothed proportion of confidence > 0.9 per day for each location,
    highlighting specified locations.
    
    Args:
        df: DataFrame with columns 'bg_datetime', 'confidence', 'location'.
        species_name: Optional species name for the plot title.
        locations: List of locations to highlight in color; others will appear in grey.
        window_size: Number of days for the rolling window (default: 5).
    """
    # Ensure datetime format and extract day
    df['bg_datetime'] = pd.to_datetime(df['bg_datetime'])
    df['day'] = df['bg_datetime'].dt.date
    df = df[(df["scaled_fg_amplitude"] > min_amplitude) &
            (df["scaled_fg_amplitude"] < max_amplitude)]

    # Define color palette
    colorblind_palette = ["#648FFF", "#FFB000", "#DC267F", "#785EF0", "#FE6100"]

    location_colors = {loc: colorblind_palette[i % len(colorblind_palette)] for i, loc in enumerate(locations)}
    default_grey = "#B0B0B0"

    if separated:
        plt.figure(figsize=(1, 10))
    elif presentation:
        plt.figure(figsize=(9, 9))
    else:
        plt.figure(figsize=(12, 8))
        
    other_plotted = False

    location_cv_list = []

    y_axis_iterator = 0.1

    location_ranges = {}
    # Step 1: Compute smoothed series and range per location    
    for location, group in df.groupby('location'):
        group = group.copy()

        # Create daily DataFrame with counts
        daily_counts = group.groupby('day')['confidence'].agg(
            total='count',
            above_threshold=lambda x: (x > confidence_threshold).sum()
        )

        # Sort by date
        daily_counts = daily_counts.sort_index()

        # Apply rolling window to get sum over the window, then compute correct proportion
        rolling_above = daily_counts['above_threshold'].rolling(window=window_size, min_periods=5).sum()
        rolling_total = daily_counts['total'].rolling(window=window_size, min_periods=5).sum()
        smoothed = rolling_above / rolling_total

        # Store the range (max - min) ignoring NaNs
        location_ranges[location] = smoothed.max() - smoothed.min()

    # Step 2: Sort locations by descending range
    sorted_locations = sorted(location_ranges, key=lambda loc: location_ranges[loc], reverse=False)
    
    # Group by location
    # Step 3: Plot in order of largest to smallest range
    for location in sorted_locations:
        group = df[df['location'] == location].copy()

        # Create daily DataFrame with counts
        daily_counts = group.groupby('day')['confidence'].agg(
            total='count',
            above_threshold=lambda x: (x > confidence_threshold).sum()
        )

        # Sort by date
        daily_counts = daily_counts.sort_index()

        # Apply rolling window to get sum over the window, then compute correct proportion
        rolling_above = daily_counts['above_threshold'].rolling(window=window_size, min_periods=5).sum()
        rolling_total = daily_counts['total'].rolling(window=window_size, min_periods=5).sum()
        smoothed = rolling_above / rolling_total

        location_cv_list.append((location, round(smoothed.min(), 3),
                                round(smoothed.max(), 3),
                                round(smoothed.max() - smoothed.min(), 3)))

        # Compute standard error and 95% confidence interval
        stderr = np.sqrt((smoothed * (1 - smoothed)) / rolling_total)
        conf_interval = 1.96 * stderr  # 95% confidence interval


        # Assign color and label
        color = location_colors.get(location, default_grey)
        label = location if location in locations else ("Other Locations" if not other_plotted else None)
        if location not in locations:
            other_plotted = True


        # Plot
        plt.plot(
            daily_counts.index,
            smoothed - smoothed.min() + y_axis_iterator if separated else smoothed,
            color=color,
            linewidth=2,
            label=label,
            alpha=1 if location in locations else 0.3,
            zorder = 5 if location in locations else 1
        )

        # Add shaded error region around each line for highlighted locations only
        if location in locations:
            if not separated:
                plt.fill_between(
                    daily_counts.index,
                    smoothed - conf_interval,
                    smoothed + conf_interval,
                    color=color,
                    alpha=0.2,
                    zorder=0
                )

        if separated:
            y_axis_iterator += (smoothed.max() - smoothed.min()) + 0.1

    if not separated:
        plt.grid(zorder=-10, alpha=0.2)
        plt.ylim(0, 1)
        plt.tight_layout()

    else:
        plt.ylim(0, y_axis_iterator)

    if presentation:
        plt.rcParams.update({'font.size': 15}) 

    # Final touches
    plt.xlabel('Date')
    plt.ylabel(f'Detectability at {confidence_threshold} confidence threshold')

    if not presentation:
        plt.title(f"Smoothed Daily Proportion > {confidence_threshold} Confidence by Location for {species_name if species_name else 'All Species'} with {min_amplitude} < fg_amp < {max_amplitude}")

    if not no_legend:
        plt.legend(title='Location', bbox_to_anchor=(1.05, 1), loc='upper left')

    plt.xticks(rotation=45)
    plt.tight_layout()

    plt.show()

    # Convert to DataFrame
    df_cv = pd.DataFrame(location_cv_list, columns=["location", "min", "max", "diff"])

    # Add rankings (1 = lowest, ascending order)
    df_cv["min_rank"] = df_cv["min"].rank(method='min', ascending=True).astype(int)
    df_cv["max_rank"] = df_cv["max"].rank(method='min', ascending=True).astype(int)
    df_cv["diff_rank"] = df_cv["diff"].rank(method='min', ascending=True).astype(int)
    df_cv["species_name"] = species_name

    ranges = [maxv - minv for _, minv, maxv, _ in location_cv_list]
    ranges = np.array(ranges)
    mean_range = ranges.mean()
    sem = ranges.std(ddof=1) / np.sqrt(len(ranges))

    print(f"mean range: {mean_range}")    

    return df_cv, location_ranges

# Create plots

In [ ]:
for species_name in names:
    filtered_df = df[(df["scaled_fg_amplitude"] > -30) &
                     (df["scaled_fg_amplitude"] < -20)]

    sites_violinplots(filtered_df[filtered_df["species_name"] == species_name], species_name = species_name, 
                      locations = locations)

In [ ]:
for species_name in df["species_name"].unique():
    plot_confidence_thresholds_by_location(df[df["species_name"] == species_name], species_name = species_name,
    locations = locations)

In [ ]:
fg_amplitude_limits = [(-40, -30)]

df_cv_list = []
stats_list = []

for species_name in df["species_name"].unique():
    for fg_amplitude_limit in fg_amplitude_limits:
        df_cv, location_ranges = plot_detectability_by_day(df[df["species_name"] == species_name], species_name = species_name,
        locations = locations, confidence_threshold = 0.9,
        min_amplitude=fg_amplitude_limit[0], max_amplitude=fg_amplitude_limit[1],
        window_size=10)

        stats_list.append((species_name, location_ranges))

        if fg_amplitude_limit[0] == -40:
            df_cv_list.append(df_cv)

df_cvs = pd.concat(df_cv_list)

In [ ]:
fg_amplitude_limits = [(-30, -20), (-50, -40)]

for species_name in df["species_name"].unique():
    for fg_amplitude_limit in fg_amplitude_limits:
        plot_confidence_thresholds_by_location_bg(df[df["species_name"] == species_name], species_name = species_name,
        locations = locations, confidence_thresholds=[0.9], 
        min_fg_amplitude=fg_amplitude_limit[0], max_fg_amplitude=fg_amplitude_limit[1],
        xlim = (-75, -10))
